In [1]:
import spacy
import numpy as np
import pandas as pd
from rasa_nlu.model import Interpreter

In [2]:
nlp = spacy.load('en')

In [3]:
interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


In [4]:
def run_nlu(query):
    global interpreter
    return interpreter.parse(query)

In [5]:
def bot():
    print('********** Chatbot Loaded! Start typing your queries here **********')
    query = ''
    while(True):
        query = input()
        query = query.lower()
        if (query=='exit'):
            break
        parsed_query = run_nlu(query)
        response = route(parsed_query)
        print(response)

In [6]:
def route(parsed_query):
    response = 'Kindly rephrase your words'
    intent = parsed_query['intent']['name']
    if (intent == 'requirement'):
        response = requirement(parsed_query)
    else if (intent == 'extent'):
        response = extent(parsed_query)
    return response

In [11]:
def extent(parsed_query):
    response = 'Kindly rephrase your words'
    extent_routings = pd.read_excel('routings/extent.xlsx')
    return 'hello'

In [9]:
def requirement(parsed_query):
    global nlp
    response = 'Kindly rephrase your words'
    requirement_routings = pd.read_excel('routings/requirement.xlsx')
    entities = parsed_query['entities']
    
    if (len(entities) == 0):
        return response
    
    status = []
    for entity in entities:
        if (entity['entity'] == 'status'):
            status.append(entity['value'])
    if (len(status) == 0):
        print('Please enter a status card that you hold')
        flag = 0
        while(flag == 0):
            status_query = input().lower()
            status_query_entities = run_nlu(status_query)['entities']
            if (len(status_query_entities) == 0):
                print('Kindly rephrase your words and enter a valid status')
            else:
                status_detected = status_query_entities[0]['value']
                flag = 1   
        status.append(status_detected)

    status = status[0]
    data = requirement_routings[requirement_routings['status'].isnull() == False]   
    required_row = data[data['status'] == status]
    if (len(required_row) == 0):
        return 'Sorry! No such information found.'
    response = required_row['response'][0]
    return response

In [10]:
bot()

********** Chatbot Loaded! Start typing your queries here **********
hi
Kindly rephrase your words
do i need a visa 
Please enter a status card that you hold
i am an nri
NRI does not require visa to visit India
i am an nri do i need a visa
NRI does not require visa to visit India
exit
